Logs 
- [2025/08/09]     
  Apply VQE to solve portfolio optimization problem

In [ ]:
imp

Objective 
$$
  \argmin_{y_{\ell, i}} \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + \frac{1}{2}
    \left(\sum_{\ell\in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell} 
    (\alpha_{\ell,i} y_{\ell,i}) (\alpha_{\ell, j} y_{\ell,j}) \right)
  + C
$$

$$
  = \argmin_{y_{\ell, i}}
    \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + 
  \frac{1}{2} \left(\sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \alpha_i^2 y_i^2
  + \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell \setminus \{i\}}
    2 \alpha_i \alpha_j y_i y_j \right)
  + C
$$

Constraints

`trade_num_upper.limit`
$$
  \left(\sum_{\ell \in L}  \sum_{i \in \mathbb{K}_\ell} y_{\ell, i} \right) \leq M
$$

`limit_sodraw.filterLevel1_sodraw.filterLevel3_#_fund_enriched.pmv`

$$
  K_{\ell}^\text{min} \leq 
    \sum_{i \in \mathbb{K}_{\ell}} \beta_{\ell, i} y_{\ell, i}
    \leq K_{\ell}^\text{max}, \qquad \forall \ell \in L
$$

`limt_sodrtaw.filterLevel1_sodraw.filterLevel3_#_fund_enriched.dxsCtr`

$$
  O_{\ell}^\text{min} \leq 
    \sum_{i \in \mathbb{K}_{\ell}} \varepsilon_{\ell, i} y_{\ell, i}
    \leq O_{\ell}^\text{max}, \qquad \forall \ell \in L
$$

`low_cnstr_cash`

$$
  P^\text{min} \leq \sum_{\ell \in L}  \sum_{i \in \mathbb{K}_\ell}
    \beta_{\ell, i} y_{\ell, i} \leq P^\text{max}
$$

$y_{\ell, i}$ are binary variables

$$
\begin{align*}
  L &= \{1, 2, 3\}  \\
    &= \{\text{Capital Goods}, \text{Insurance}, \text{Transporation}\}
\end{align*}
$$

$$
  \mathbb{K}_{\ell}
  = \begin{cases}
      \begin{aligned}
      \mathbb{K}_1 &= \{1, 2, 3, \ldots, 12\} 
        = \left\{
        \begin{array}{l}
        \texttt{020002BJ9}, \texttt{026874DS3}, \texttt{15135BAW1}, 
        \texttt{21871XAS8}, \texttt{444859BR2}, \\ \texttt{444859BV3}, 
        \texttt{444859BY7}, \texttt{444859CA8}, \texttt{540424AT5}, 
        \texttt{56501RAN6}, \\ \texttt{759351AP4}, \texttt{91324PEJ7} 
        \end{array}
        \right\},
      \end{aligned} \\[16pt]
      \mathbb{K}_2 = \{1, 2, 3, \ldots, 13\}
        = \left\{
        \begin{array}{l}
          \texttt{081437AT2}, \texttt{097023CJ2}, \texttt{14448CBC7}, 
          \texttt{24422EWZ8}, \texttt{24422EXP9}, \\ \texttt{36166NAK9}, 
          \texttt{438516CM6}, \texttt{443201AC2}, \texttt{45687VAB2}, 
          \texttt{539830CD9}, \\ \texttt{75513EAD3}, \texttt{760759BA7}, 
          \texttt{760759BC3}
        \end{array}
        \right\}, \\[16pt]
      \mathbb{K}_3 = \{1, 2, 3, \ldots, 6\}
        = \left\{
        \begin{array}{l}
          \texttt{13645RAD6}, \texttt{13645RBF0}, \texttt{314353AA1}, 
          \texttt{655844CR7}, \texttt{655844CT3}, \\ \texttt{907818FX1}
        \end{array}
        \right\}
    \end{cases}
$$

$$
  M = \texttt{trade\_num\_upper\_limit} = 500
$$